**Train Exactly As Hard As You Should (TEAHAYS) Beta**

Start by entering your macrocycle data. You may use pounds or kilograms, but please be consistent with your units throughout.

Note: data for Competition Squat, Competition Bench Press, and Competition Deadlift will be graphed below. If you would like those graphs to appear with meaningful data, please enter data for those lifts.

In [ ]:
import ipywidgets as widgets
import datetime
import pandas as pd
import altair as alt
import numpy as np
import matplotlib.pyplot as plot
data = []

dtpick = widgets.DatePicker(
 description='Date',
 disabled = False,
 value = datetime.date.today()
 )
display(dtpick)
    
txtsl = widgets.Text(
  placeholder='Competition Squat',
  description='Lift'
  )
display(txtsl)
    
weight = widgets.FloatText(
 value=100,
 description='Weight'
 )
display(weight)
    
Reps = widgets.IntText(
  value=5,
  description='Reps'
  )
display(Reps)
   
RPEslider = widgets.IntSlider(
 value=7, # default value
 min=0, # min value
 max=10, # max value
 step=1, # increment size
 description='RPE' # slider label
 )
display(RPEslider) # display the slider

DatePicker(value=datetime.date(2021, 6, 3), description='Date')

Text(value='', description='Lift', placeholder='Competition Squat')

FloatText(value=100.0, description='Weight')

IntText(value=5, description='Reps')

IntSlider(value=7, description='RPE', max=10)

In [ ]:
btn = widgets.Button(description='Enter workout!')
display(btn)
def btn_eventhandler(obj):
    print('Workout added!'.format(obj.description))
    data.append([dtpick.value.isoformat(), txtsl.value, weight.value, Reps.value - RPEslider.value + 10, Reps.value, RPEslider.value])
btn.on_click(btn_eventhandler)

Button(description='Enter workout!', style=ButtonStyle())

Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout added!
Workout ad

Here's your data:

In [ ]:
df = pd.DataFrame(data, columns = ['Date','Lift','Weight','Projected Reps','Reps','RPE'])
display(df)

,Date,Lift,Weight,Projected Reps,Reps,RPE
0,2018-11-28,Competition Squat,155.0,15,5,0
1,2018-11-28,Competition Deadlift,185.0,15,5,0
2,2018-11-28,Good Morning,115.0,15,5,0
3,2018-12-04,Competition Squat,185.0,10,5,5
4,2018-12-04,Competition Deadlift,240.0,10,5,5
...,...,...,...,...,...,...
91,2018-11-20,Competition Deadlift,270.0,10,10,10
92,2018-09-07,Dumbbell Row,40.0,20,10,0
93,2018-09-14,Dumbbell Row,50.0,15,10,5
94,2018-09-21,Dumbbell Row,55.0,11,10,9


In [ ]:
lifts = list(df['Lift'].unique())
liftdfs = dict()
for lift in lifts:
    liftdfs[lift] = df[df.Lift == lift]
print(len(liftdfs))
for compLift in ['Competition Squat','Competition Bench Press','Competition Deadlift']:
    series = df['Lift'].str.contains(compLift)
    found = False
    for x in series:
        if x:
            found = True
    if not found:
        liftdfs[compLift] = pd.DataFrame([[datetime.date.today().isoformat(),compLift,1,1,1,1]], columns = ['Date','Lift','Weight','Projected Reps','Reps','RPE'])
        lifts.append(compLift)

12


We will now calculate a cubic polynomial of best fit for each lift, including accessories. We choose a cubic polynomial because it's the perfect balance between under and overfitting, and it's easy to get a polynomial that is increasing as reps decrease. We graph the trend curves for each of your competition lifts.

In [ ]:
liftModels = dict()
liftbasegraphs = dict()
liftpolygraphs = dict()

for x in lifts:
    base = alt.Chart(liftdfs[x],title=x).mark_circle().encode(
            alt.X("Projected Reps",sort='descending'), alt.Y("Weight")
    )

    polynomial = base.transform_regression(
            "Projected Reps", "Weight", method="poly", order=3, as_=["Projected Reps", str(3)]
        )


    polynomial_fit = [polynomial.mark_line().transform_fold([str(3)], as_=['degree',"Weight"])]

    X = liftdfs[x].iloc[:,3].values
    Y = liftdfs[x].iloc[:,2].values
    mymodel = np.poly1d(np.polyfit(X,Y,3))
    liftModels[x] = mymodel
    liftbasegraphs[x] = base
    liftpolygraphs[x] = polynomial_fit

    
squat = alt.layer(liftbasegraphs["Competition Squat"], *liftpolygraphs["Competition Squat"])
dl = alt.layer(liftbasegraphs["Competition Deadlift"], *liftpolygraphs["Competition Deadlift"])
bench = alt.layer(liftbasegraphs["Competition Bench Press"], *liftpolygraphs["Competition Bench Press"])
alt.hconcat(squat,bench,dl)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)


alt.HConcatChart(...)

Now we can use the polynomial of best fit to determine what weight you should lift for a given numbers of reps and given RPE in order to achieve your next competition goal. If you use a similar macrocycle as last time, this should be a good weight for you. After you do your workout, come back and enter your new workout data at the top of the page (though you need to keep the page open, for now). We'll adjust your trend curves as you go. If you're always missing your lifts, your goal is too high. If your workouts are too easy, your goal is too low.

In [ ]:
txtsl1 = widgets.Text(
  placeholder='What lift to set goal for?',
  description='Lift'
  )
display(txtsl1)
weight1 = widgets.FloatText(
 value=100,
 description='New 1RM Goal'
 )
display(weight1)

Text(value='', description='Lift', placeholder='What lift to set goal for?')

FloatText(value=100.0, description='New 1RM Goal')

In [ ]:
lift = txtsl1.value
goal = weight1.value
value = liftModels[lift](1)
ratio = float(goal)/float(value)
goalmodel = liftModels[lift] * ratio
print('Now what training set do you have today?')
Reps1 = widgets.IntText(
  value=5,
  description='Reps'
  )
display(Reps1)
RPEslider1 = widgets.IntSlider(
 value=7, # default value
 min=0, # min value
 max=10, # max value
 step=1, # increment size
 description='RPE' # slider label
 )
display(RPEslider1) # display the slider

Now what training set do you have today?


IntText(value=5, description='Reps')

IntSlider(value=7, description='RPE', max=10)

In [ ]:
reps = Reps1.value
rpe = RPEslider1.value
projectedreps = reps - rpe + 10
print("You should lift "+str(goalmodel(projectedreps))+" for "+str(reps)+" at RPE "+str(rpe)+" on "+lift+".")

You should lift 270.27656090736696 for 5 at RPE 10 on Competition Squat.


You can track your Projected 1RM for any lift over time, to see if you're making progress. This tells you whether your current training is effective for that lift.

In [ ]:
txtsl2 = widgets.Text(
  placeholder='Lift to track progress for',
  description='Lift'
  )
display(txtsl2)

Text(value='', description='Lift', placeholder='Lift to track progress for')

In [ ]:
lift = txtsl2.value
liftdfs[lift]['Projected 1RM'] =  liftModels[lift](1) * liftdfs[lift]['Weight']/liftModels[lift](liftdfs[lift]['Projected Reps'])
small = liftdfs[lift]['Projected 1RM'].min()
big = liftdfs[lift]['Projected 1RM'].max()
alt.Chart(liftdfs[lift], title=lift).mark_circle().encode(
        alt.X('Date:T', timeUnit='yearmonthdate', title = 'Date'), alt.Y("Projected 1RM", scale = alt.Scale(domain = [small-10,big+10])), alt.Color("RPE")
)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


alt.Chart(...)

**Decide if an accessory lift is useful for a main lift**

You can plot your Projected 1RM for an accessory against a main lift to see if that accessory is helping your main lift. If the red line is sloping downward, you might consider swapping out that accessory.

In [ ]:
dropD = widgets.Dropdown(
 options=['Competition Squat','Competition Bench Press','Competition Deadlift'],
 description='Main Lift',
 disabled=False,
 )
display(dropD)

txtsl3 = widgets.Text(
  placeholder='Accessory',
  description='Accessory'
  )
display(txtsl3)

Dropdown(description='Main Lift', options=('Competition Squat', 'Competition Bench Press', 'Competition Deadli…

Text(value='', description='Accessory', placeholder='Accessory')

In [ ]:
mainLift = dropD.value
accessory = txtsl3.value

liftdfs[mainLift]['Projected 1RM'] =  liftModels[mainLift](1) * liftdfs[mainLift]['Weight']/liftModels[mainLift](liftdfs[mainLift]['Projected Reps'])
liftdfs[accessory]['Projected 1RM'] =  liftModels[accessory](1) * liftdfs[accessory]['Weight']/liftModels[accessory](liftdfs[accessory]['Projected Reps'])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
combineddf = liftdfs[mainLift]
extradf=liftdfs[accessory]
newDf = combineddf.merge(extradf,how='outer',on='Date',suffixes=(' Main',' Acc'))
fixedDf = newDf.interpolate(method='linear',limit_direction='forward',axis=0)

In [ ]:
smallx = fixedDf['Projected 1RM Main'].min()
bigx = fixedDf['Projected 1RM Main'].max()
smally = fixedDf['Projected 1RM Acc'].min()
bigy = fixedDf['Projected 1RM Acc'].max()

base = alt.Chart(fixedDf,title=mainLift+' versus '+accessory).mark_circle().encode(
            alt.X("Projected 1RM Main", title="Projected 1RM for "+mainLift, scale=alt.Scale(domain = [smallx-10,bigx+10])), alt.Y("Projected 1RM Acc", title="Projected 1RM for "+accessory, scale = alt.Scale(domain=[smally-10,bigy+10])))


line = base.transform_regression("Projected 1RM Main", "Projected 1RM Acc",method="linear"
).mark_line(color="red")

alt.layer(base,line)

alt.LayerChart(...)

In [2]:
import nbinteract as nbi
nbi.publish('cse512-21s/FP-Powerlifting', 'Train_Exactly_As_Hard_As_You_Should.ipynb')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saving notebook... Saved 'Train_Exactly_As_Hard_As_You_Should.ipynb'.
Converting notebook...



Successfully converted!

<a href="Train_Exactly_As_Hard_As_You_Should.html" target="_blank" download>Click to download your webpage.</a>

To host your webpage, see the documentation:
<a href="https://www.nbinteract.com/tutorial/tutorial_publishing.html"
        target="_blank">
    https://www.nbinteract.com/tutorial/tutorial_publishing.html
</a>
